In [9]:
import os
from dotenv import load_dotenv

# Load environment variables from openai.env file
load_dotenv("openai.env")
# Read the OPENAI_API_KEY from the environment
api_key = os.getenv("OPENAI_API_KEY")
api_base = os.getenv("OPENAI_API_BASE")
# os.environ["OPENAI_API_KEY"] = api_key
# os.environ["OPENAI_API_BASE"] = api_base

# ChatDoc:又一个智能文档助手

- 读取pdf、excel、doc三种常见的文档格式
- 根据文档内容，智能抽取内容并输出相应格式
<hr>

In [ ]:
#安装必须的包
# 处理doc文档的包
! pip install docx2txt
# 处理pdf的包
! pip install pypdf
! pip install nltk

## 文档加载切割

In [7]:
#导入必须的包
from langchain.document_loaders import UnstructuredExcelLoader, Docx2txtLoader, PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter


#定义chatdoc
class ChatDoc():
    def __init__(self, doc):
        self.doc = doc
        self.splitText = []  #分割后的文本

    # 文档加载
    def getFile(self):
        doc = self.doc
        loaders = {
            "docx": Docx2txtLoader,
            "pdf": PyPDFLoader,
            "xlsx": UnstructuredExcelLoader,
        }
        file_extension = doc.split(".")[-1]
        loader_class = loaders.get(file_extension)
        if loader_class:
            try:
                loader = loader_class(doc)
                text = loader.load()
                return text
            except Exception as e:
                print(f"Error loading {file_extension} files:{e}")
                return None
        else:
            print(f"Unsupported file extension: {file_extension}")
            return None

    #处理文档的函数
    def splitSentences(self):
        full_text = self.getFile()  #获取文档内容
        if full_text != None:
            #对文档进行分割
            text_split = CharacterTextSplitter(
                chunk_size=150,
                chunk_overlap=20,
            )
            texts = text_split.split_documents(full_text)
            self.splitText = texts


chat_doc = ChatDoc(doc="testfile/loader.docx")
chat_doc.splitSentences()
chat_doc.splitText

[Document(page_content='一、公司基本信息\n\n名称：宏图科技发展有限公司\n\n注册地址：江苏省南京市雨花台区软件大道101号\n\n成立日期：2011年5月16日\n\n法定代表人：李强\n\n注册资本：人民币5000万元\n\n员工人数：约200人\n\n联系电话：025-88888888\n\n电子邮箱：info@hongtutech.cn', metadata={'source': 'testfile/loader.docx'}),
 Document(page_content='二、财务状况概述\n\n截至2023年第一季度，宏图科技发展有限公司财务状况堪忧，具体情况如下：\n\n1. 资产总额：人民币1.2亿元，较上年同期下降30%。\n\n2. 负债总额：人民币1.8亿元，较上年同期上升50%，资不抵债。\n\n3. 营业收入：人民币3000万元，较上年同期下降60%。', metadata={'source': 'testfile/loader.docx'}),
 Document(page_content='4. 净利润：亏损人民币800万元，去年同期为盈利人民币200万元。\n\n5. 现金流量：公司现金流量紧张，现金及现金等价物余额为人民币500万元，难以支撑日常运营。\n\n6. 存货：存货积压严重，库存商品价值约为人民币400万元，大部分产品滞销。', metadata={'source': 'testfile/loader.docx'}),
 Document(page_content='7. 应收账款：应收账款高达人民币600万元，回收难度大，坏账准备不足。\n\n三、主营业务及市场状况\n\n宏图科技发展有限公司主要从事计算机软件的研发与销售。近年来，由于市场竞争加剧、技术更新换代速度快和管理层决策失误等原因，公司主营业务收入持续下降。目前，公司面临的主要问题有：', metadata={'source': 'testfile/loader.docx'}),
 Document(page_content='1. 产品同质化严重，缺乏核心竞争力。\n\n2. 新产品开发进度缓慢，未能及时抓住市场需求变化。\n\n3. 市场营销策略不当，导致市场份额大幅缩水。\n\n4. 行业内新兴企业崛起迅速，原有客户流失严重

## 向量化与向量存储

In [12]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings  #向量化与向量存储


#向量化与向量存储
def embedding_and_vectordb(chat_doc):
    embeddings = OpenAIEmbeddings()
    db = Chroma.from_documents(
        documents=chat_doc.splitText,
        embedding=embeddings,
    )
    return db


emb = embedding_and_vectordb(chat_doc);
print(emb);

## 索引并使用自然语言找出相关的文本块

In [15]:
 #提问并找到相关的文本块
def ask_and_find_files(chat_doc, question):
    db = embedding_and_vectordb(chat_doc)
    retriever = db.as_retriever()
    results = retriever.invoke(question)
    return results


chat_doc = ChatDoc(doc="testfile/loader.docx")
chat_doc.splitSentences()
ask_and_find_files(chat_doc, "这家公司叫什么名字?")
# 可以看到检索结果不精准,返回了很多, 所以需要优化检索,提高检索精确度

[Document(page_content='一、公司基本信息\n\n名称：宏图科技发展有限公司\n\n注册地址：江苏省南京市雨花台区软件大道101号\n\n成立日期：2011年5月16日\n\n法定代表人：李强\n\n注册资本：人民币5000万元\n\n员工人数：约200人\n\n联系电话：025-88888888\n\n电子邮箱：info@hongtutech.cn', metadata={'source': 'testfile/loader.docx'}),
 Document(page_content='一、公司基本信息\n\n名称：宏图科技发展有限公司\n\n注册地址：江苏省南京市雨花台区软件大道101号\n\n成立日期：2011年5月16日\n\n法定代表人：李强\n\n注册资本：人民币5000万元\n\n员工人数：约200人\n\n联系电话：025-88888888\n\n电子邮箱：info@hongtutech.cn', metadata={'source': 'testfile/loader.docx'}),
 Document(page_content='一、公司基本信息\n\n名称：宏图科技发展有限公司\n\n注册地址：江苏省南京市雨花台区软件大道101号\n\n成立日期：2011年5月16日\n\n法定代表人：李强\n\n注册资本：人民币5000万元\n\n员工人数：约200人\n\n联系电话：025-88888888\n\n电子邮箱：info@hongtutech.cn', metadata={'source': 'testfile/loader.docx'}),
 Document(page_content='一、公司基本信息\n\n名称：宏图科技发展有限公司\n\n注册地址：江苏省南京市雨花台区软件大道101号\n\n成立日期：2011年5月16日\n\n法定代表人：李强\n\n注册资本：人民币5000万元\n\n员工人数：约200人\n\n联系电话：025-88888888\n\n电子邮箱：info@hongtutech.cn', metadata={'source': 'testfile/loader.docx'})]

# 检索优化

## 使用多重查询提高文档检索精确度

In [18]:
from langchain.chat_models import ChatOpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever


def ask_and_find_files(chat_doc, question):
    db = embedding_and_vectordb(chat_doc)
    retriever = db.as_retriever()
    #把问题交给LLM进行多角度的扩展
    llm = ChatOpenAI(temperature=0)
    retriever_from_llm = MultiQueryRetriever.from_llm(
        retriever=retriever,
        llm=llm,
    )
    return retriever_from_llm.get_relevant_documents(question)


chat_doc = ChatDoc(doc="testfile/loader.docx")
chat_doc.splitSentences()
#设置下logging查看生成查询
import logging

logging.basicConfig(level=logging.INFO)
# 大模型会从不同角度将该问题扩展为多个不同的问题,然后从检索会获得多个答案 ,然后会选择多个答案中重复出现的答案作为最终结果返回
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.DEBUG)
ask_and_find_files(chat_doc, "这家公司叫什么名字?")
# 可以看到检索结果不精准,返回了很多, 所以需要优化检索,提高检索精确度

INFO:httpx:HTTP Request: POST https://api.aihubmix.com/v1/embeddings "HTTP/1.1 200 OK"
/Users/jianghui/PycharmProjects/ai_agent_study/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
INFO:httpx:HTTP Request: POST https://api.aihubmix.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:langchain.retrievers.multi_query:Generated queries: ['1. 请问这个企业的名称是什么？', '2. 你知道这家公司的名字是什么吗？', '3. 可以告诉我这个公司的名称吗？']
INFO:httpx:HTTP Request: POST https://api.aihubmix.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.aihubmix.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP

[Document(page_content='一、公司基本信息\n\n名称：宏图科技发展有限公司\n\n注册地址：江苏省南京市雨花台区软件大道101号\n\n成立日期：2011年5月16日\n\n法定代表人：李强\n\n注册资本：人民币5000万元\n\n员工人数：约200人\n\n联系电话：025-88888888\n\n电子邮箱：info@hongtutech.cn', metadata={'source': 'testfile/loader.docx'})]

## 使用上下文压缩检索降低冗余信息
1. 先使用向量数据库检索
2. 把问题和结果调用大模型做压缩,剔除不相关的内容,然后返回

![111](contextual_compression.png)

In [23]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.chat_models import ChatOpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever


def ask_and_find_files(chat_doc, question):
    db = embedding_and_vectordb(chat_doc)
    retriever = db.as_retriever()
    #把问题交给LLM进行多角度的扩展
    llm = ChatOpenAI(temperature=0)
    compressor = LLMChainExtractor.from_llm(llm=llm)
    compressor_retriever = ContextualCompressionRetriever(base_retriever=retriever, base_compressor=compressor)
    return compressor_retriever.get_relevant_documents(query=question)


chat_doc = ChatDoc(doc="testfile/loader.docx")
chat_doc.splitSentences()
ask_and_find_files(chat_doc, "这家公司注册地点在哪里?")
# ask_and_find_files(chat_doc, "这家公司负债多少?")

INFO:httpx:HTTP Request: POST https://api.aihubmix.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.aihubmix.com/v1/embeddings "HTTP/1.1 200 OK"
/Users/jianghui/PycharmProjects/ai_agent_study/venv/lib/python3.11/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
INFO:httpx:HTTP Request: POST https://api.aihubmix.com/v1/chat/completions "HTTP/1.1 200 OK"
/Users/jianghui/PycharmProjects/ai_agent_study/venv/lib/python3.11/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
INFO:httpx:HTTP Request: POST https://api.aihubmix.com/v1/chat/completions "HTTP/1.1 200 OK"
/Users/jianghui/PycharmProjects/ai_agent_study/venv/lib/python3.11/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, i

[Document(page_content='注册地址：江苏省南京市雨花台区软件大道101号', metadata={'source': 'testfile/loader.docx'}),
 Document(page_content='注册地址：江苏省南京市雨花台区软件大道101号', metadata={'source': 'testfile/loader.docx'}),
 Document(page_content='注册地址：江苏省南京市雨花台区软件大道101号', metadata={'source': 'testfile/loader.docx'}),
 Document(page_content='注册地址：江苏省南京市雨花台区软件大道101号', metadata={'source': 'testfile/loader.docx'})]

## 在向量存储里使用最大边际相似性（MMR）和相似性打分

In [26]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.chat_models import ChatOpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever


def ask_and_find_files(chat_doc, question):
    db = embedding_and_vectordb(chat_doc)
    retriever = db.as_retriever(search_type="mmr")
    # retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": .1, "k": 1})
    return retriever.get_relevant_documents(query=question)


chat_doc = ChatDoc(doc="testfile/loader.docx")
chat_doc.splitSentences()
# ask_and_find_files(chat_doc, "这家公司注册地点在哪里?")
ask_and_find_files(chat_doc, "这家公司负债多少?")

INFO:httpx:HTTP Request: POST https://api.aihubmix.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.aihubmix.com/v1/embeddings "HTTP/1.1 200 OK"


[Document(page_content='1. 银行贷款：公司向多家银行贷款总额达人民币1亿元，部分贷款已逾期未还。\n\n2. 供应商欠款：因现金流紧张，公司拖欠供应商货款达人民币300万元。\n\n3. 员工工资及社保：由于资金链断裂，公司拖欠员工工资及社保费用共计人民币200万元。', metadata={'source': 'testfile/loader.docx'}),
 Document(page_content='1. 银行贷款：公司向多家银行贷款总额达人民币1亿元，部分贷款已逾期未还。\n\n2. 供应商欠款：因现金流紧张，公司拖欠供应商货款达人民币300万元。\n\n3. 员工工资及社保：由于资金链断裂，公司拖欠员工工资及社保费用共计人民币200万元。', metadata={'source': 'testfile/loader.docx'}),
 Document(page_content='1. 银行贷款：公司向多家银行贷款总额达人民币1亿元，部分贷款已逾期未还。\n\n2. 供应商欠款：因现金流紧张，公司拖欠供应商货款达人民币300万元。\n\n3. 员工工资及社保：由于资金链断裂，公司拖欠员工工资及社保费用共计人民币200万元。', metadata={'source': 'testfile/loader.docx'}),
 Document(page_content='4. 净利润：亏损人民币800万元，去年同期为盈利人民币200万元。\n\n5. 现金流量：公司现金流量紧张，现金及现金等价物余额为人民币500万元，难以支撑日常运营。\n\n6. 存货：存货积压严重，库存商品价值约为人民币400万元，大部分产品滞销。', metadata={'source': 'testfile/loader.docx'})]

# 和文件聊天

- 先使用本地向量数据库选择出相关的上下文
- 将问题和相关上下文传递给大模型,大模型会根据上下文返回答案

In [30]:
#导入必须的包
from langchain.document_loaders import UnstructuredExcelLoader, Docx2txtLoader, PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
#导入聊天所需的模块
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate


#定义chatdoc
class ChatDoc():
    def __init__(self, doc):
        self.doc = doc
        self.splitText = []  #分割后的文本
        self.template = [
            ("system",
             "你是一个处理文档的秘书,你从不说自己是一个大模型或者AI助手,你会根据下面提供的上下文内容来继续回答问题.\n 上下文内容\n {context} \n"),
            ("human", "你好！"),
            ("ai", "你好"),
            ("human", "{question}"),
        ]
        self.prompt = ChatPromptTemplate.from_messages(self.template)

    def getFile(self):
        doc = self.doc
        loaders = {
            "docx": Docx2txtLoader,
            "pdf": PyPDFLoader,
            "xlsx": UnstructuredExcelLoader,
        }
        file_extension = doc.split(".")[-1]
        loader_class = loaders.get(file_extension)
        if loader_class:
            try:
                loader = loader_class(doc)
                text = loader.load()
                return text
            except Exception as e:
                print(f"Error loading {file_extension} files:{e}")
                return None
        else:
            print(f"Unsupported file extension: {file_extension}")
            return None

    #处理文档的函数
    def splitSentences(self):
        full_text = self.getFile()  #获取文档内容
        if full_text != None:
            #对文档进行分割
            text_split = CharacterTextSplitter(chunk_size=150, chunk_overlap=20)
            texts = text_split.split_documents(full_text)
            self.splitText = texts

    #向量化与向量存储
    def embeddingAndVectorDB(self):
        embeddings = OpenAIEmbeddings()
        db = Chroma.from_documents(documents=self.splitText, embedding=embeddings)
        return db

    #提问并找到相关的文本块
    def askAndFindFiles(self, question):
        db = self.embeddingAndVectorDB()
        retriever = db.as_retriever(search_type="similarity_score_threshold",
                                    search_kwargs={"score_threshold": .5, "k": 1})
        return retriever.get_relevant_documents(query=question)

    #用自然语言和文档聊天
    def chatWithDoc(self, question):
        _content = ""
        context = self.askAndFindFiles(question)
        for i in context:
            _content += i.page_content

        messages = self.prompt.format_messages(context=_content, question=question)
        logging.info(messages)
        chat = ChatOpenAI(model="gpt-4", temperature=0)
        return chat.invoke(messages)


chat_doc = ChatDoc(doc="testfile/loader.docx")
chat_doc.splitSentences()
# chat_doc.chatWithDoc("公司注册地址是哪里？")
# chat_doc.chatWithDoc("公司盈利了吗?")
chat_doc.chatWithDoc("公司上市了吗?")


INFO:httpx:HTTP Request: POST https://api.aihubmix.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.aihubmix.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:root:[SystemMessage(content='你是一个处理文档的秘书,你从不说自己是一个大模型或者AI助手,你会根据下面提供的上下文内容来继续回答问题.\n 上下文内容\n 4. 净利润：亏损人民币800万元，去年同期为盈利人民币200万元。\n\n5. 现金流量：公司现金流量紧张，现金及现金等价物余额为人民币500万元，难以支撑日常运营。\n\n6. 存货：存货积压严重，库存商品价值约为人民币400万元，大部分产品滞销。 \n'), HumanMessage(content='你好！'), AIMessage(content='你好'), HumanMessage(content='公司上市了吗?')]
INFO:httpx:HTTP Request: POST https://api.aihubmix.com/v1/chat/completions "HTTP/1.1 200 OK"


AIMessage(content='对不起，根据提供的信息，我无法确定公司是否已经上市。您是否有更多的信息可以提供？', response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 213, 'total_tokens': 247}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-79fdf554-f03f-45fd-8105-202d1252f60f-0')